In [ ]:
import math

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.datasets import mnist
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

import pandas as pd 
import numpy as np 
import sklearn 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelBinarizer



In [ ]:
NUM_CLASSES = 7

In [ ]:
path = "data.csv" 
 
df = pd.read_csv(path)
print(df)

# New Section

In [ ]:
Y = df['label'].values.reshape(df.shape[0],1)
df = df.drop('sec', 1)
df = df.drop('label', 1)
dataset = df.values 
X = dataset[:,0:dataset.shape[1]]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2 ) # training to testing ratio is 0.8:0.2 

print("x_train.shape = {}, y_train.shape = {}".format(x_train.shape, y_train.shape))
print("x_test.shape = {}, y_test.shape = {}".format(x_test.shape, y_test.shape))

#print(y_train)

encoder = LabelBinarizer()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
#print(transformed_label)
print(y_train.shape)

#y_train = to_categorical(y_train, 3)
#y_test = to_categorical(y_test, 3)
#print("y_train.shape = {}, y_test.shape = {}".format(y_train.shape, y_test.shape))



In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, input_shape=(7,), activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3, activation="softmax")
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
lr_decay = lambda epoch: 0.0001 + 0.02 * math.pow(1.0 / math.e, epoch / 3.0)
decay_callback = LearningRateScheduler(lr_decay, verbose=1)

history = model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, 
                    validation_data=(x_test, y_test), callbacks=[decay_callback])

In [ ]:
model.save('model/mydata.h5')

In [ ]:
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file('model/mydata.h5')
tflite_model = converter.convert()
open('mydata.tflite', 'wb').write(tflite_model)

In [ ]:
try:
    from google.colab import files
    files.download('mydata.tflite')
except:
    print("Skip downloading")